In [ ]:
import torch
import numpy as np
from sklearn.datasets import load_diabetes
from 

X , y = load_diabetes(return_X_y=True)



X_tensor = torch.Tensor(X)
y_tensor = torch.Tensor(y)



device(type='cpu')